# Zusammenbringen der Datensätze simRa und osm_surface

Unser Vorhersage-Modell soll anhand von Features wie z.B. Straßentyp oder Oberflächenbeschaffenheit Vorhersagen zum Gefahrenpotentail einer Stecke machen.   
Dazu müssen wir den SimRa-Datensatz und unsere Features aus OSM vereinen.

#### Das Vorgehen entspricht dem dieses notebooks:  [osm_highway](osm_highway.ipynb)  
genauere Erklärungen finden sich im angezeigten notebook

In [2]:
import geopandas as gpd


# Lade die GeoJSON-Datei
osm_surface = gpd.read_file("../../data/processed_data/cycle_net_berlin_cleaned_surface.geojson")

In [3]:
simra_data = gpd.read_file("../../data/processed_data/simra_within_berlin.geojson")

In [4]:
# Sicherstellen, dass die Geometriespalten korrekt gesetzt sind
simra_data = simra_data.set_geometry('geometry')
osm_surface = osm_surface.set_geometry('geometry')

#### Überprüfung der Koordinatensysteme beider Datensätze

In [5]:
simra_data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
osm_surface.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

#### Verknüpfen der Datensätze mit Join

Verwenden des left Joins:

Behalten aller Einträge aus dem ersten (linken) DataFrame (simra) und hinzufügen passender Einträge aus dem zweiten (rechten) DataFrame (osm_surface)Einträge im linken DataFrame ohne passende Einträge im rechten DataFrame erhalten NaN-Werte für die Spalten aus dem rechten DataFrame.

In [7]:
# Räumliche Verknüpfung zwischen Polygondaten und den osm/fahrradnetzwerk/highway-Daten
#simRa_surface = gpd.sjoin(simra_data, osm_surface, how='left', op='intersects')
# "op" Parameter bald nicht mehr gültig, sattdessen soll "predicate" benutzt werden

# Räumliche Verknüpfung zwischen Polygondaten und den OSM/Fahrradnetzwerk/Highway-Daten
simRa_surface = gpd.sjoin(simra_data, osm_surface, how='left', predicate='intersects')


In [8]:
simRa_surface.head()

,id,type,score,incidents,rides,markers,geometry,index_right,surface_category
0,[79310].0,Street,0.000000,0,57,[ ],"POLYGON ((13.37410 52.53031, 13.37421 52.53020...",170.0,asphalt
0,[79310].0,Street,0.000000,0,57,[ ],"POLYGON ((13.37410 52.53031, 13.37421 52.53020...",33845.0,asphalt
0,[79310].0,Street,0.000000,0,57,[ ],"POLYGON ((13.37410 52.53031, 13.37421 52.53020...",145435.0,asphalt
1,"[196724641, 196725586, 866264912].0",Junction,0.000649,1,1541,"[ [ [ 13.417860660000001, 52.514469009999999 ]...","POLYGON ((13.41751 52.51461, 13.41779 52.51442...",13371.0,asphalt
1,"[196724641, 196725586, 866264912].0",Junction,0.000649,1,1541,"[ [ [ 13.417860660000001, 52.514469009999999 ]...","POLYGON ((13.41751 52.51461, 13.41779 52.51442...",13373.0,asphalt


### Zuordnung mehrerer surface-Typen zu einem Polyglon/Segment 

Gründe für Mehrfachzuordung:  
*(aus osm_highway notebook)*

1. Wenn ein Polygon aus dem simra_data mehrere Straßenabschnitte im osm_surface überlappt oder einschließt, wird jedes dieser überlappenden Segmente in der räumlichen Verknüpfung berücksichtigt, was zur Mehrfachzuordnung führt.
2. OSM-Daten enthalten detaillierte Informationen, bei denen Straßen in einzelne Segmente unterteilt werden können (z.B. bei Kreuzungen, Änderungen der Straßentypen oder Fahrspuren). Daher kann ein einzelnes Polygon mehrere dieser Segmente einschließen.Räumliche Beziehungen:
3. Die räumliche Verknüpfung (predicate='intersects') berücksichtigt alle Geometrien, die im rechten DataFrame das Polygon im linken DataFrame schneiden. Das bedeutet, dass mehrere Straßenabschnitte demselben Polygon zugeordnet werden können, wenn sie sich schneiden bzw. überlappen.

**TODO:** 
Es muss entschieden werden wie mit den mehrfachen Zuordnungen umgegangen werden soll

### Überprüfen der NaN Werte

die Prüfung ergibt eine Übereinstimmung mit den Daten bezüglich der highways (siehe notebook [osm_highway](osm_highway.ipynb))  

Im weiteren wird so verfahren wie beim joinen des SimRa Datensatzes und des osm_highway Datensatzes.   
Nähere Erläuterungen sin d aus dem "osm_highway notebook" zu entnehmen.


In [9]:
# Zeilen, die NaN in 'surface' (bzw. in 'index_right') enthalten
nan_values = simRa_surface[simRa_surface['surface_category'].isna()]

print(f"Anzahl der Zeilen mit NaN in 'surface_category': {len(nan_values)}")

Anzahl der Zeilen mit NaN in 'surface_category': 376


In [10]:
nan_values_score = simRa_surface[(simRa_surface['surface_category'].isna()) & (simRa_surface['score'] != 0)]

In [11]:
nan_values_score.count()

id                  63
type                63
score               63
incidents           63
rides               63
markers             63
geometry            63
index_right          0
surface_category     0
dtype: int64

In [12]:
nan_values_score.explore()


### Entfernen der NaN-Werte

Nähere Begründung siehe [osm_highway](osm_highway.ipynb)  

Bereinigung der Daten  
Um sicherzustellen, dass nur relevante Verknüpfungen in unserer Analyse betrachtet werden, entfernen wir Zeilen mit NaN-Werten in der indexright-Spalte. Das bedeutet, dass nur Zeilen mit gefundenen räumlichen Übereinstimmungen beibehalten werden.

In [13]:
cleaned_simra_surface = simRa_surface.dropna(subset=['surface_category'])

print(f"Anzahl der verbleibenden Zeilen nach dem Entfernen der NaN-Werte: {len(cleaned_simra_surface)}")

Anzahl der verbleibenden Zeilen nach dem Entfernen der NaN-Werte: 85801


In [14]:
# Sicherstellen, dass keine NaN-Werte mehr in der Spalte 'index_right' vorhanden sind
nan_values_after_cleaning = cleaned_simra_surface[cleaned_simra_surface['index_right'].isna()]

print(f"Anzahl der NaN-Werte nach der Bereinigung: {len(nan_values_after_cleaning)}")

Anzahl der NaN-Werte nach der Bereinigung: 0


In [15]:
cleaned_simra_surface.shape

(85801, 9)

In [16]:
cleaned_simra_surface.head()

,id,type,score,incidents,rides,markers,geometry,index_right,surface_category
0,[79310].0,Street,0.000000,0,57,[ ],"POLYGON ((13.37410 52.53031, 13.37421 52.53020...",170.0,asphalt
0,[79310].0,Street,0.000000,0,57,[ ],"POLYGON ((13.37410 52.53031, 13.37421 52.53020...",33845.0,asphalt
0,[79310].0,Street,0.000000,0,57,[ ],"POLYGON ((13.37410 52.53031, 13.37421 52.53020...",145435.0,asphalt
1,"[196724641, 196725586, 866264912].0",Junction,0.000649,1,1541,"[ [ [ 13.417860660000001, 52.514469009999999 ]...","POLYGON ((13.41751 52.51461, 13.41779 52.51442...",13371.0,asphalt
1,"[196724641, 196725586, 866264912].0",Junction,0.000649,1,1541,"[ [ [ 13.417860660000001, 52.514469009999999 ]...","POLYGON ((13.41751 52.51461, 13.41779 52.51442...",13373.0,asphalt


#### Die Betrachtung der Ergebnisse zeigt, dass die Ausgaben (NaN Werte und deren Geometrien) identisch sind mit dem Join von simRa und dem Highway-Datensatz.  



### Gruppieren der Polyglone und Zusammenfassen der surface-Werte

Gruppieren und Aggregieren:  

* Der DataFrame wird nach id und geometry gruppiert. Für jede Gruppe werden die angegebenen Aggregationsfunktionen auf die Spalten angewendet:
  * Für type, score, incidents, rides, markers, und index_right wird der erste Wert in der Gruppe verwendet ('first').
  * Für die surface_category-Spalte wird eine Funktion zum Kombinieren der Werte angwendet.
* Das Ergebnis ist ein DataFrame grouped_data, bei dem die maxspeed-Werte pro Gruppe als zusammenhängender String dargestellt sind, während alle anderen spezifischen Werte beibehalten werden.

In [17]:
cleaned_simra_surface = gpd.GeoDataFrame(cleaned_simra_surface, geometry='geometry')

In [18]:
# Funktion zum Kombinieren der 'highway'-Werte 
def combine_surfaces(x):
    return ', '.join(x)  # Doppelte Einträge bleiben erhalten und werden verbunden

# Gruppieren nach 'id' und 'geometry' und Aggregation
grouped_data = cleaned_simra_surface.groupby(['id', 'geometry']).agg({
    'type': 'first',       # Erster Wert (da alle Werte gleich)
    'score': 'first',      
    'incidents': 'first',  
    'rides': 'first',      
    'markers': 'first',    
    'index_right': 'first',
    'surface_category': combine_surfaces  # Kombinieren der 'highway' Werte
}).reset_index()

In [19]:
# Umwandlung zurück in ein GeoDataFrame
grouped_data = gpd.GeoDataFrame(grouped_data, geometry='geometry', crs=cleaned_simra_surface.crs)

In [20]:
grouped_data.head(3)

,id,geometry,type,score,incidents,rides,markers,index_right,surface_category
0,[100049].0,"POLYGON ((13.45412 52.54035, 13.45320 52.53977...",Street,0.0,0,138,[ ],57832.0,"concrete, concrete, concrete, concrete"
1,[100069498].0,"POLYGON ((13.52273 52.50704, 13.52248 52.50690...",Junction,0.0,0,200,[ ],123443.0,"asphalt, asphalt, asphalt"
2,"[100078509, 288268004, 3888645535].0","POLYGON ((13.47754 52.51457, 13.47782 52.51438...",Junction,0.0,0,54,[ ],75145.0,"asphalt, asphalt, asphalt, asphalt, asphalt, a..."


In [21]:
# Beispielausgabe surface_category bei INdex 72
grouped_data.loc[72, 'surface_category']

'asphalt, unpaved, asphalt, asphalt, unpaved, paving_stone, unpaved, paving_stone, asphalt, unpaved'

In [22]:
grouped_data.shape


(15918, 9)

shape des Datensatzes ist identisch zum Datensatz simRa-Hihgway  
Das ist positiv.

### Umgang mit mehreren surface-Werten in einem Polygon
-------

surface Werte könnten anteilig  nach ihrem Vorkommen im Polygon ausgegeben werden  
Diesen Ansatz wollen wir verfolgen und später testen, ob dieses Vorgehen sinnvoll ist

Wir gehen folgendermaßen vor:  
* Zerlegen der surface_category-Spalte.
* Berechnung der Anteile für jeden surface-Typ.
* Erstellung der neuen Kategorien-Spalten.

In [23]:
# Funktion zum Zerlegen der 'surface_category'-Spalte in Listen
grouped_data['surface_list'] = grouped_data['surface_category'].apply(lambda x: x.split(', '))

In [24]:
grouped_data.head()

,id,geometry,type,score,incidents,rides,markers,index_right,surface_category,surface_list
0,[100049].0,"POLYGON ((13.45412 52.54035, 13.45320 52.53977...",Street,0.0,0,138,[ ],57832.0,"concrete, concrete, concrete, concrete","[concrete, concrete, concrete, concrete]"
1,[100069498].0,"POLYGON ((13.52273 52.50704, 13.52248 52.50690...",Junction,0.0,0,200,[ ],123443.0,"asphalt, asphalt, asphalt","[asphalt, asphalt, asphalt]"
2,"[100078509, 288268004, 3888645535].0","POLYGON ((13.47754 52.51457, 13.47782 52.51438...",Junction,0.0,0,54,[ ],75145.0,"asphalt, asphalt, asphalt, asphalt, asphalt, a...","[asphalt, asphalt, asphalt, asphalt, asphalt, ..."
3,[100094].0,"POLYGON ((13.46855 52.61490, 13.46841 52.61475...",Street,0.0,0,98,[ ],6529.0,"asphalt, unpaved","[asphalt, unpaved]"
4,[1000].0,"POLYGON ((13.35533 52.51693, 13.35655 52.51683...",Street,0.0,0,130,[ ],156528.0,"asphalt, asphalt, asphalt","[asphalt, asphalt, asphalt]"


In [25]:
# Alle einzigartigen `surface`-Typen finden
unique_types = sorted(set(sum(grouped_data['surface_list'].tolist(), [])))

In [26]:
unique_types

['asphalt', 'concrete', 'paving_stone', 'sett', 'unpaved']

In [27]:


# Funktion zur Berechnung der Anteilswerte - berechnet die Anteile der jeweiligen `surface`-Typen pro Zeile
def calculate_surface_ratios(row, surface_types):
    total_count = len(row['surface_list'])
    counts = pd.Series(row['surface_list']).value_counts()
    return {surface: counts.get(surface, 0) / total_count for surface in surface_types}

In [28]:
import pandas as pd

# Anwendung der Funktion auf den GeoDataFrame
surface_ratios = grouped_data.apply(calculate_surface_ratios, axis=1, surface_types=unique_types) # axis=1 --> Fkt. wird aus Zeilen angewendet
ratios_df = pd.DataFrame(list(surface_ratios))

In [29]:
# Zusammenführen der Ergebnisse mit dem ursprünglichen GeoDataFrame
gdf = pd.concat([grouped_data, ratios_df], axis=1)

In [30]:
# Entfernen der temporären Spalte
gdf.drop(columns=['surface_list'], inplace=True)

In [31]:
# Entfernen der temporären Spalte
gdf.drop(columns=['surface_category', 'markers'], inplace=True)

In [32]:
gdf.head(10)

,id,geometry,type,score,incidents,rides,index_right,asphalt,concrete,paving_stone,sett,unpaved
0,[100049].0,"POLYGON ((13.45412 52.54035, 13.45320 52.53977...",Street,0.000000,0,138,57832.0,0.000000,1.0,0.000000,0.0,0.000000
1,[100069498].0,"POLYGON ((13.52273 52.50704, 13.52248 52.50690...",Junction,0.000000,0,200,123443.0,1.000000,0.0,0.000000,0.0,0.000000
2,"[100078509, 288268004, 3888645535].0","POLYGON ((13.47754 52.51457, 13.47782 52.51438...",Junction,0.000000,0,54,75145.0,0.846154,0.0,0.000000,0.0,0.153846
3,[100094].0,"POLYGON ((13.46855 52.61490, 13.46841 52.61475...",Street,0.000000,0,98,6529.0,0.500000,0.0,0.000000,0.0,0.500000
4,[1000].0,"POLYGON ((13.35533 52.51693, 13.35655 52.51683...",Street,0.000000,0,130,156528.0,1.000000,0.0,0.000000,0.0,0.000000
5,[100120].0,"POLYGON ((13.50803 52.45148, 13.50766 52.45048...",Street,0.000000,0,54,46806.0,0.500000,0.5,0.000000,0.0,0.000000
6,[100126].0,"POLYGON ((13.50823 52.45280, 13.50841 52.45267...",Street,0.000000,0,169,46808.0,1.000000,0.0,0.000000,0.0,0.000000
7,[100129].0,"POLYGON ((13.50832 52.45291, 13.50806 52.45301...",Street,0.000000,0,102,46808.0,1.000000,0.0,0.000000,0.0,0.000000
8,[100144].0,"POLYGON ((13.40858 52.51198, 13.40761 52.51167...",Street,0.001818,1,550,46823.0,1.000000,0.0,0.000000,0.0,0.000000
9,[100154846].0,"POLYGON ((13.51004 52.45376, 13.50972 52.45359...",Junction,0.000000,0,414,37257.0,0.833333,0.0,0.166667,0.0,0.000000


In [33]:
 #Speichern des gdf

output_filename = "../../data/processed_data/osm_surface_ratios.geojson"
gdf.to_file(output_filename, driver='GeoJSON')

print(f"Datei erfolgreich gespeichert.")

Datei erfolgreich gespeichert.
